In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random


In [ ]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']
# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
gene_num=(df_proceed['Metadata_gene'].value_counts())
gene_num_list=gene_num.index.tolist()
# Only in list of plate is for mAP classifiation, so we filter it out
lst=['BR00116991','BR00116992','BR00116993','BR00116994','BR00117015','BR00117016','BR00117017','BR00117019','BR00116995','BR00117024','BR00117025','BR00117026','BR00117010','BR00117011','BR00117012','BR00117013']

# 8:2 Split Target Gene
random.seed(1)
train_lst=random.sample(gene_num_list, 104)#####
test_lst = []
for m in gene_num_list:
    if m not in train_lst:
        test_lst.append(m)
        
df_compound=df_proceed[df_proceed['Metadata_experiment_type']=='Compound']
df_compound.to_csv('/data/datacenter/H3C_GPU/projects/yuchen/gzsda-main/gzsda-main/data/XrayBaggage20/total_compound.csv')
# 80% target gene of Compound + Gene perturbation 

dftrain1=df_compound[df_compound['Metadata_gene'].isin(train_lst)]  

# 100% CRISPR for Train
dftrain2=df_proceed[df_proceed['Metadata_experiment_type']=='CRISPR']    
dftrain2.to_csv('/data/datacenter/H3C_GPU/projects/yuchen/gzsda-main/gzsda-main/data/XrayBaggage20/total_CRISPR.csv')
# To meet the mAP evaluation Plate
train_part_total=pd.DataFrame()
for i in lst:
    train_part_total=pd.concat([train_part_total,dftrain1])#.loc[dftrain1['Metadata_Plate']==i,:]])
train_part0=pd.DataFrame()
for i in ['BR00116991','BR00116992','BR00116993','BR00116994']:
    train_part0=pd.concat([train_part0,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part1=pd.DataFrame()
for i in ['BR00117015','BR00117016','BR00117017','BR00117019']:
    train_part1=pd.concat([train_part1,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part2=pd.DataFrame()
for i in ['BR00116995','BR00117024','BR00117025','BR00117026']:
    train_part2=pd.concat([train_part2,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part3=pd.DataFrame()
for i in ['BR00117010','BR00117011','BR00117012','BR00117013']:
    train_part3=pd.concat([train_part3,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])  
    
# test data sampling
gene_seen=pd.DataFrame()
for i in train_lst:
    train_part0_filtered=train_part0.loc[train_part0['Metadata_gene']==i,:]
    train_part1_filtered=train_part1.loc[train_part1['Metadata_gene']==i,:]
    train_part2_filtered=train_part2.loc[train_part2['Metadata_gene']==i,:]
    train_part3_filtered=train_part3.loc[train_part3['Metadata_gene']==i,:]

# To filter seen/unseen gene from compound perturbation

    if len(train_part0_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part0_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part0_filtered[train_part0_filtered['Metadata_pert_iname']==filter_out]])
        
    if len(train_part1_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part1_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part1_filtered[train_part1_filtered['Metadata_pert_iname']==filter_out]]) 
        
    if len(train_part2_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part2_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part2_filtered[train_part2_filtered['Metadata_pert_iname']==filter_out]])  
        
    if len(train_part3_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part3_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part3_filtered[train_part3_filtered['Metadata_pert_iname']==filter_out]])
train_part_total_final=train_part_total.drop(index=list(gene_seen['Unnamed: 0'].index))
train_part_total_final=train_part_total_final.drop_duplicates()
df_test1=gene_seen
df_test1=df_test1[df_test1['Metadata_Plate'].isin(lst)]
# train
df_train=pd.concat([train_part_total_final,dftrain2]) 
df_test2=df_proceed[df_proceed['Metadata_gene'].isin(test_lst)]
df_test2=df_test2.drop_duplicates()
df_test2=df_test2[df_test2['Metadata_experiment_type']=='Compound']
df_test2=df_test2[df_test2['Metadata_Plate'].isin(lst)]
# SEEN

# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
df_test1_rm_emptygene=df_test1[df_test1['Metadata_gene']!='empty']
y_test=df_test1_rm_emptygene[df_test1_rm_emptygene['Metadata_experiment_type']=='Compound']['Metadata_gene']
testX=df_test1_rm_emptygene[df_test1_rm_emptygene['Metadata_experiment_type']=='Compound'].drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)
df_proceed = df_proceed[df_proceed['Metadata_Plate'].isin(lst)]

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, recall_score

# Scale the training and test sets using StandardScaler
scaler = StandardScaler()
trainX_scaled = scaler.fit_transform(trainX)
testX_scaled = scaler.transform(testX)

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, recall_score

# Scale the training and test sets using StandardScaler
scaler = StandardScaler()
trainX_scaled = scaler.fit_transform(trainX)
testX_scaled = scaler.transform(testX)

# Train the MLPClassifier model
mlp_clf = MLPClassifier(hidden_layer_sizes=(1280, 630, 300),
                        max_iter=30, activation='relu', verbose=True, tol=1e-8,
                        solver='adam')
mlp_clf.fit(trainX_scaled, trainY)

These below are MLP Classification

In [ ]:
# plug in mode = 'top_1' if you want to apply normal MLP classification
# plug in mode = 'top_10' if you want to get the top 10 highest 
def Compute_MLP_Result(trainX,testX,testY,trainY,mode):
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score
    import numpy as np
    from sklearn.neural_network import MLPClassifier
    from sklearn.metrics import classification_report, recall_score
    if mode == 'top_1':
    # MLP TOP ONE (Not TOP TEN)
        sc=StandardScaler()
        scaler = sc.fit(trainX)
        trainX_scaled = scaler.transform(trainX)
        testX_scaled = scaler.transform(testX)
        y_pred = mlp_clf.predict(testX_scaled)
    #print('Seen')
    #print(classification_report(testY, y_pred))
        report=classification_report(testY, y_pred,output_dict=True)
        result=pd.DataFrame(report).transpose()[['recall']]
        result.rename(columns={'recall': 'Recall'},inplace=True)
        result.drop(['micro avg','macro avg','weighted avg'],axis=0,errors='ignore',inplace = True)
        result.index.name='Label'
        result=result[result.index.isin(pd.Series(testY).unique())]
        return result
    if mode =='top_10':
        # MLP Top Ten

        from sklearn.preprocessing import LabelEncoder
        sc = StandardScaler()
        sc.fit(testX)
        le = LabelEncoder()
        trainY_encoded = le.fit_transform(trainY)
        testY_encoded = le.transform(testY)
        y_pred_probs = mlp_clf.predict_proba(sc.transform(testX))

# Initialize a dictionary to keep track of the number of correct predictions for each label
        label_counts = {label: {'total': 0, 'correct': 0} for label in set(testY)}

# Loop over each sample in the test set
        for i in range(len(testY)):
            true_label = testY[i]
            true_label_encoded = testY_encoded[i]
            top_ten_labels_encoded = np.argsort(y_pred_probs[i])[::-1][:10]
            top_ten_labels = le.inverse_transform(top_ten_labels_encoded)
            if true_label in top_ten_labels:
                label_counts[true_label]['correct'] += 1
            label_counts[true_label]['total'] += 1

# Calculate the accuracy for each label and print the results
        for label, counts in label_counts.items():
            if counts['total'] > 0:
                accuracy = counts['correct'] / counts['total']
                print(f"Label {label}: {accuracy:.2f} ({counts['correct']}/{counts['total']})")
            else:
                print(f"Label {label}: No samples in test set")

# Calculate the accuracy for each label and store the results in a dictionary
        accuracy_dict = {}
        for label, counts in label_counts.items():
            if counts['total'] > 0:
                accuracy = counts['correct'] / counts['total']
                accuracy_dict[label] = accuracy

# Create a DataFrame with the accuracy for each label
        accuracy_df = pd.DataFrame.from_dict(accuracy_dict, orient='index', columns=['Accuracy'])
        accuracy_df.index.name = 'Label'
        accuracy_df = accuracy_df.sort_values(by='Accuracy', ascending=False)
        return accuracy_df
        

In [ ]:
# top1 / Top 10 Result Seen
top1_seen_result=Compute_MLP_Result(trainX,testX,testY,trainY,'top_1')
top10_seen_result=Compute_MLP_Result(trainX,testX,testY,trainY,'top_10')

In [ ]:
#### UNSEEN data Preparasion
dfte2_rmempty=df_test2[df_test2['Metadata_gene']!='empty']
y_test=dfte2_rmempty[dfte2_rmempty['Metadata_experiment_type']=='Compound']['Metadata_gene']
testX=dfte2_rmempty[dfte2_rmempty['Metadata_experiment_type']=='Compound'].drop('Metadata_gene', axis = 1)
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX_final=testX
testY_final=y_test
testX=np.array(testX)
testY=np.array(y_test)

In [ ]:
# top1 / Top 10 Result Unseen
top1_unseen_result=Compute_MLP_Result(trainX,testX,testY,trainY,'top_1')
top10_unseen_result=Compute_MLP_Result(trainX,testX,testY,trainY,'top_10')

In [ ]:
#Summary of top 1 result MLP
top1_seen_result['If_seen']='seen'
top1_unseen_result['If_seen']='unseen'
all_top1_result=pd.concat([top1_seen_result,top1_unseen_result])
all_top1_result=all_top1_result.dropna()
print('Top1 AVG Seen ACC MLP',top1_seen_result['Recall'].mean())
print('Top1 AVG Unseen ACC MLP',top1_unseen_result['Recall'].mean())
print('Top1 Harmonic Mean MLP',2/((1/top1_seen_result['Recall'].mean())+ 1 /top1_unseen_result['Recall'].mean() ))

In [ ]:
#Summary of top 10 result MLP
top10_seen_result['If_seen']='seen'
top10_unseen_result['If_seen']='unseen'
top10_all_result=pd.concat([top10_seen_result,top10_unseen_result])
top10_all_result=top10_all_result.dropna()
print('Top10 AVG Seen ACC MLP',top10_seen_result['Accuracy'].mean())
print('Top10 AVG Unseen ACC MLP',top10_unseen_result['Accuracy'].mean())
print('Top10 Harmonic Mean MLP',2/((1/top10_seen_result['Accuracy'].mean())+ 1 /top10_unseen_result['Accuracy'].mean() ))

In [ ]:
##### Feature Transformation for Test Set Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=pd.concat([df_test1,df_test2]).iloc[:,pd.concat([df_test1,df_test2]).columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=pd.concat([df_test1,df_test2]).iloc[:,0:pd.concat([df_test1,df_test2]).columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,pd.concat([df_test1,df_test2])['Metadata_experiment_type']], axis=1)

In [ ]:
feature_new=feature_new.iloc[:,1:-1]
compound_new=feature_new[feature_new['Metadata_experiment_type']=='Compound']
compound_new=compound_new[compound_new['Metadata_Plate'].isin(lst)]

In [ ]:
compound_new.to_csv('/data/datacenter/H3C_GPU/projects/yuchen/gzsda-main/gzsda-main/data/XrayBaggage20/Compound.csv')

In [ ]:
df_cris=df_train[df_train['Metadata_experiment_type']=='CRISPR']
##### Feature Transformation for Train CRISPR 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_cris.iloc[:,df_cris.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_cris.iloc[:,0:df_cris.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
CRISPR_new =pd.concat([transformen_seen,feature,df_cris['Metadata_experiment_type']], axis=1)

In [ ]:
CRISPR_new.to_csv('/data/datacenter/H3C_GPU/projects/yuchen/gzsda-main/gzsda-main/data/XrayBaggage20/CRISPR.csv')

In [ ]:
feature_new=feature_new.reset_index()
df_test1=df_test1.reset_index()
df_test2=df_test2.reset_index()
df_test1=df_test1.iloc[:,2:]
df_test2=df_test2.iloc[:,2:]
feature_new=feature_new.iloc[:,2:]

In [ ]:
##### Feature Transformation Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_test1.iloc[:,df_test1.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_test1.iloc[:,0:df_test1.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_test1['Metadata_experiment_type']], axis=1)
dfte1a=feature_new
dfte1a.to_csv('MLP_seen_ccvae.csv')

In [ ]:
##### Feature Transformation Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_test2.iloc[:,df_test2.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_test2.iloc[:,0:df_test2.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_test2['Metadata_experiment_type']], axis=1)
dfte2a=feature_new
dfte2a.to_csv('MLP_unseen_ccvae.csv')

These below is for 1-NN Classification

In [ ]:
##### Feature Transformation Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_train.iloc[:,df_train.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_train.iloc[:,0:df_train.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_train['Metadata_experiment_type']], axis=1)
dftrain_new=feature_new

# Modify the shape of data with transformed features
y_train=dftrain_new['Metadata_gene']
trainX=dftrain_new.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc(0):]
from sklearn.neighbors import KNeighborsClassifier
y_test=dfte1a['Metadata_gene']
testX=dfte1a.drop('Metadata_gene', axis = 1)
testX=testX.iloc[:,testX.columns.get_loc(0):]
testX=testX.iloc[:,:-1]

## prepare for training
trainX=trainX.iloc[:,:-1]
trainX=np.array(trainX)
trainY=y_train.to_list()
trainY=np.array(trainY)
testX=np.array(testX)
testY=np.array(y_test)
k=130
label_vectors=trainX

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

def kNCM(trainX, trainY, testX, testY, k, mode):
    # Convert the label strings to integers
    label_to_int = {label: i for i, label in enumerate(set(trainY))}
    trainY_int = np.array([label_to_int[label] for label in trainY])
    testY_int = np.array([label_to_int[label] for label in testY])

    # Compute the mean vectors for each class in the training set
    class_means = {}
    for label in set(trainY_int):
        class_means[label] = np.mean(trainX[trainY_int == label], axis=0)

    # Use the class means as the label vectors
    label_vectors = np.array(list(class_means.values()))

    # Classify the test samples using the k-nearest class mean algorithm
    knn_clf = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    knn_clf.fit(label_vectors, list(class_means.keys()))
    nn_indices = knn_clf.kneighbors(testX, return_distance=False)

    # Compute the predicted labels for each test sample
    top_ten_labels_int = np.array([[knn_clf.classes_[i] for i in nn] for nn in nn_indices])

    # Convert the label indices to label names and filter out invalid labels
    int_to_label = {i: label for label, i in label_to_int.items()}
    top_ten_labels = np.vectorize(lambda x: int_to_label.get(x, None))(top_ten_labels_int)
    if mode == 'top_1':
        top_ten_labels = np.array([labels[labels != None][:1] for labels in top_ten_labels])
    if mode == 'top_10':
        top_ten_labels = np.array([labels[labels != None][:10] for labels in top_ten_labels])

    # Merge testY and top_ten_labels into a DataFrame
    df = pd.DataFrame({'testY': testY, 'top_ten_labels': top_ten_labels.tolist()})
    lst=[]
    for i in range(len(df)):
        if df['testY'][i] in df['top_ten_labels'][i]:
            score=1
        else:
            score=0
        lst.append(score)
    df['score']=lst
    avg_df = df.groupby('testY')['score'].mean().reset_index()
    avg_df=avg_df.sort_values(by='score',ascending= False)
    return avg_df

In [ ]:
KNCM_seen_result= kNCM(trainX, trainY, testX, testY, k,'top_10')
KNCM_seen_result['If_seen']='seen'
KNCM_seen_result1= kNCM(trainX, trainY, testX, testY, k,'top_1')
KNCM_seen_result1['If_seen']='seen'

In [ ]:
##### Feature Transformation Unseen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_train.iloc[:,df_train.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_train.iloc[:,0:df_train.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_train['Metadata_experiment_type']], axis=1)
dftrain_new=feature_new

##### Modify the shape and contents of data transformed new feature
y_train=dftrain_new['Metadata_gene']
trainX=dftrain_new.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc(0):]
from sklearn.neighbors import KNeighborsClassifier
y_test=dfte2a['Metadata_gene']
testX=dfte2a.drop('Metadata_gene', axis = 1)
testX=testX.iloc[:,testX.columns.get_loc(0):]
testX=testX.iloc[:,:-1]

## Prepare for training 
trainX=trainX.iloc[:,:-1]
trainX=np.array(trainX)
trainY=y_train.to_list()
trainY=np.array(trainY)
testX=np.array(testX)
testY=np.array(y_test)
k=130
label_vectors=trainX

In [ ]:
KNCM_unseen_result= kNCM(trainX, trainY, testX, testY, k,'top_10')
KNCM_unseen_result['If_seen']='unseen'
KNCM_total_result=pd.concat([KNCM_seen_result,KNCM_unseen_result])
print('MLP_KNCM_SEEN_TOP10',KNCM_seen_result['score'].mean())
print('MLP_KNCM_UNEEN_TOP10',KNCM_unseen_result['score'].mean())
print('MLP_KNCM_HARMONIC_MEAN_TOP10',2/((1/KNCM_seen_result['score'].mean())+ 1 /KNCM_unseen_result['score'].mean() ))
KNCM_unseen_result1= kNCM(trainX, trainY, testX, testY, k,'top_1')
KNCM_unseen_result1['If_seen']='unseen'
KNCM_total_result1=pd.concat([KNCM_seen_result,KNCM_unseen_result])
print('')
print('MLP_KNCM_SEEN_TOP1',KNCM_seen_result1['score'].mean())
print('MLP_KNCM_UNEEN_TOP1',KNCM_unseen_result1['score'].mean())
print('MLP_KNCM_HARMONIC_MEAN_TOP1',2/((1/KNCM_seen_result1['score'].mean())+ 1 /KNCM_unseen_result1['score'].mean() ))

This is for SLPP

In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random

# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']
# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
gene_num=(df_proceed['Metadata_gene'].value_counts())
gene_num_list=gene_num.index.tolist()
# Only in list of plate is for mAP classifiation, so we filter it out
lst=['BR00116991','BR00116992','BR00116993','BR00116994','BR00117015','BR00117016','BR00117017','BR00117019','BR00116995','BR00117024','BR00117025','BR00117026','BR00117010','BR00117011','BR00117012','BR00117013']

# 8:2 Split Target Gene
random.seed(1)
train_lst=random.sample(gene_num_list, 104)#####
test_lst = []
for m in gene_num_list:
    if m not in train_lst:
        test_lst.append(m)
        
df_compound=df_proceed[df_proceed['Metadata_experiment_type']=='Compound']
#df_compound.to_csv('/data/datacenter/H3C_GPU/projects/yuchen/gzsda-main/gzsda-main/data/XrayBaggage20/total_compound.csv')
# 80% target gene of Compound + Gene perturbation 

dftrain1=df_compound[df_compound['Metadata_gene'].isin(train_lst)]  

# 100% CRISPR for Train
dftrain2=df_proceed[df_proceed['Metadata_experiment_type']=='CRISPR']    
#dftrain2.to_csv('/data/datacenter/H3C_GPU/projects/yuchen/gzsda-main/gzsda-main/data/XrayBaggage20/total_CRISPR.csv')
# To meet the mAP evaluation Plate
train_part_total=pd.DataFrame()
for i in lst:
    train_part_total=pd.concat([train_part_total,dftrain1])#.loc[dftrain1['Metadata_Plate']==i,:]])
train_part0=pd.DataFrame()
for i in ['BR00116991','BR00116992','BR00116993','BR00116994']:
    train_part0=pd.concat([train_part0,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part1=pd.DataFrame()
for i in ['BR00117015','BR00117016','BR00117017','BR00117019']:
    train_part1=pd.concat([train_part1,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part2=pd.DataFrame()
for i in ['BR00116995','BR00117024','BR00117025','BR00117026']:
    train_part2=pd.concat([train_part2,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part3=pd.DataFrame()
for i in ['BR00117010','BR00117011','BR00117012','BR00117013']:
    train_part3=pd.concat([train_part3,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])  
    
# test data sampling
gene_seen=pd.DataFrame()
for i in train_lst:
    train_part0_filtered=train_part0.loc[train_part0['Metadata_gene']==i,:]
    train_part1_filtered=train_part1.loc[train_part1['Metadata_gene']==i,:]
    train_part2_filtered=train_part2.loc[train_part2['Metadata_gene']==i,:]
    train_part3_filtered=train_part3.loc[train_part3['Metadata_gene']==i,:]

# To filter seen/unseen gene from compound perturbation

    if len(train_part0_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part0_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part0_filtered[train_part0_filtered['Metadata_pert_iname']==filter_out]])
        
    if len(train_part1_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part1_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part1_filtered[train_part1_filtered['Metadata_pert_iname']==filter_out]]) 
        
    if len(train_part2_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part2_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part2_filtered[train_part2_filtered['Metadata_pert_iname']==filter_out]])  
        
    if len(train_part3_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part3_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part3_filtered[train_part3_filtered['Metadata_pert_iname']==filter_out]])
train_part_total_final=train_part_total.drop(index=list(gene_seen['Unnamed: 0'].index))
train_part_total_final=train_part_total_final.drop_duplicates()
df_test1=gene_seen
df_test1=df_test1[df_test1['Metadata_Plate'].isin(lst)]
# train
df_train=pd.concat([train_part_total_final,dftrain2]) 
df_test2=df_proceed[df_proceed['Metadata_gene'].isin(test_lst)]
df_test2=df_test2.drop_duplicates()
df_test2=df_test2[df_test2['Metadata_experiment_type']=='Compound']
df_test2=df_test2[df_test2['Metadata_Plate'].isin(lst)]
# SEEN

# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
df_test1_rm_emptygene=df_test1[df_test1['Metadata_gene']!='empty']
y_test=df_test1_rm_emptygene[df_test1_rm_emptygene['Metadata_experiment_type']=='Compound']['Metadata_gene']
testX=df_test1_rm_emptygene[df_test1_rm_emptygene['Metadata_experiment_type']=='Compound'].drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)
df_proceed = df_proceed[df_proceed['Metadata_Plate'].isin(lst)]

/tmp/ipykernel_3362641/3157794454.py:11: DtypeWarning: Columns (2,6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [3]:
n_components=300
sigma=1.0

In [4]:
## code from: github.com/tanyapole/reproduce-OSLPP/blob/main/OSLPP.py
import scipy 

def get_l2_norm(features:np.ndarray): return np.sqrt(np.square(features).sum(axis=1)).reshape((-1,1))

def get_l2_normalized(features:np.ndarray): return features / get_l2_norm(features)

def get_PCA(features, dim):
    result = PCA(n_components=dim).fit_transform(features)
    assert len(features) == len(result)
    return result

def get_W(labels,):
    W = (labels.reshape(-1,1) == labels).astype(np.int32)
    negative_one_idxs = np.where(labels == -1)[0]
    W[:,negative_one_idxs] = 0
    W[negative_one_idxs,:] = 0
    return W

def get_D(W): return np.eye(len(W), dtype=np.int32) * W.sum(axis=1)

def fix_numerical_assymetry(M): return (M + M.transpose()) * 0.5

def get_projection_matrix(features, labels, proj_dim):
    N, d = features.shape
    X = features.transpose()
    
    W = get_W(labels)
    D = get_D(W)
    L = D - W

    A = fix_numerical_assymetry(np.matmul(np.matmul(X, D), X.transpose()))
    B = fix_numerical_assymetry(np.matmul(np.matmul(X, L), X.transpose()) + np.eye(d))
    assert (A.transpose() == A).all() and (B.transpose() == B).all()

    w, v = scipy.linalg.eigh(A, B)
    assert w[0] < w[-1]
    w, v = w[-proj_dim:], v[:, -proj_dim:]
    assert np.abs(np.matmul(A, v) - w * np.matmul(B, v)).max() < 1e-5

    w = np.flip(w)
    v = np.flip(v, axis=1)

    for i in range(v.shape[1]):
        if v[0,i] < 0:
            v[:,i] *= -1
    return v

def project_features(P, features):
    # P: pca_dim x proj_dim
    # features: N x pca_dim
    # result: N x proj_dim
    return np.matmul(P.transpose(), features.transpose()).transpose()

y_train_array = np.array(y_train)
P = get_projection_matrix(trainX, y_train_array, n_components)

In [5]:
# Transform training data feature  from 904 to 300 dimension
transformed_data = np.dot(df_train.iloc[:,15:], P)
new_feature=pd.DataFrame(transformed_data)
trainX1=df_train.iloc[:,:df_train.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
trainX1=trainX1.reset_index()
dftrain_trans =pd.concat([trainX1,new_feature],axis=1)

In [6]:
# Transform SEEN test data feature  from 904 to 300 dimension
transformed_data = np.dot(testX, P)
new_feature=pd.DataFrame(transformed_data)
testX1=df_test1.iloc[:,:df_test1.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
testX1=testX1.reset_index()
dfnew1 =pd.concat([testX1,new_feature],axis=1)

In [7]:
# Prepare for training KNCM
y_train=dftrain_trans['Metadata_gene']
trainX=dftrain_trans.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc(0):]
from sklearn.neighbors import KNeighborsClassifier
y_test=dfnew1['Metadata_gene']
testX=dfnew1.drop('Metadata_gene', axis = 1)
testX=testX.iloc[:,testX.columns.get_loc(0):]
trainY=y_train.tolist()
testY=y_test.tolist()

In [8]:
k=130
KNCM_seen_result= kNCM(trainX, trainY, testX, testY, k,'top_10')
KNCM_seen_result['If_seen']='seen'
KNCM_seen_result1= kNCM(trainX, trainY, testX, testY, k,'top_1')
KNCM_seen_result1['If_seen']='seen'

In [9]:
## This is for Unseen Test data preparasion
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
df_test2_rm_emptygene=df_test2[df_test2['Metadata_gene']!='empty']
y_test=df_test2_rm_emptygene[df_test2_rm_emptygene['Metadata_experiment_type']=='Compound']['Metadata_gene']
testX=df_test2_rm_emptygene[df_test2_rm_emptygene['Metadata_experiment_type']=='Compound'].drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)
X_train = trainX
y_train = trainY

In [10]:
## feature transformation on Unseen test set
transformed_data = np.dot(testX, P)
new_feature=pd.DataFrame(transformed_data)
testX2=df_test2.iloc[:,:df_test2.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
testX2=testX2.reset_index()
dfnew2 =pd.concat([testX2,new_feature],axis=1)

In [11]:
## Data Prepare for training
y_train=dftrain_trans['Metadata_gene']
trainX=dftrain_trans.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc(0):]
from sklearn.neighbors import KNeighborsClassifier
y_test=dfnew2['Metadata_gene']
testX=dfnew2.drop('Metadata_gene', axis = 1)
testX=testX.iloc[:,testX.columns.get_loc(0):]
trainY=y_train.tolist()
testY=y_test.tolist()

In [12]:
KNCM_unseen_result= kNCM(trainX, trainY, testX, testY, k,'top_10')
KNCM_unseen_result['If_seen']='unseen'
KNCM_total_result=pd.concat([KNCM_seen_result,KNCM_unseen_result])
print('SLPP_KNCM_SEEN_TOP10',KNCM_seen_result['score'].mean())
print('SLPP_KNCM_UNEEN_TOP10',KNCM_unseen_result['score'].mean())
print('SLPP_KNCM_HARMONIC_MEAN_TOP10',2/((1/KNCM_seen_result['score'].mean())+ 1 /KNCM_unseen_result['score'].mean() ))
KNCM_unseen_result1= kNCM(trainX, trainY, testX, testY, k,'top_1')
KNCM_unseen_result1['If_seen']='unseen'
KNCM_total_result1=pd.concat([KNCM_seen_result1,KNCM_unseen_result1])
print('')
print('SLPP_KNCM_SEEN_TOP1',KNCM_seen_result1['score'].mean())
print('SLPP_KNCM_UNEEN_TOP1',KNCM_unseen_result1['score'].mean())
print('SLPP_KNCM_HARMONIC_MEAN_TOP1',2/((1/KNCM_seen_result1['score'].mean())+ 1 /KNCM_unseen_result1['score'].mean() ))

SLPP_KNCM_SEEN_TOP10 0.5595
SLPP_KNCM_UNEEN_TOP10 0.04233870967741936
SLPP_KNCM_HARMONIC_MEAN_TOP10 0.07872045344910758

SLPP_KNCM_SEEN_TOP1 0.22854166666666664
SLPP_KNCM_UNEEN_TOP1 0.0
SLPP_KNCM_HARMONIC_MEAN_TOP1 0.0


/tmp/ipykernel_3362641/4031413575.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  print('SLPP_KNCM_HARMONIC_MEAN_TOP1',2/((1/KNCM_seen_result1['score'].mean())+ 1 /KNCM_unseen_result1['score'].mean() ))


This is for Cellprofiler

In [ ]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']
# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
gene_num=(df_proceed['Metadata_gene'].value_counts())
gene_num_list=gene_num.index.tolist()
# Only in list of plate is for mAP classifiation, so we filter it out
lst=['BR00116991','BR00116992','BR00116993','BR00116994','BR00117015','BR00117016','BR00117017','BR00117019','BR00116995','BR00117024','BR00117025','BR00117026','BR00117010','BR00117011','BR00117012','BR00117013']

# 8:2 Split Target Gene
random.seed(1)
train_lst=random.sample(gene_num_list, 104)#####
test_lst = []
for m in gene_num_list:
    if m not in train_lst:
        test_lst.append(m)
        
df_compound=df_proceed[df_proceed['Metadata_experiment_type']=='Compound']
#df_compound.to_csv('/data/datacenter/H3C_GPU/projects/yuchen/gzsda-main/gzsda-main/data/XrayBaggage20/total_compound.csv')
# 80% target gene of Compound + Gene perturbation 

dftrain1=df_compound[df_compound['Metadata_gene'].isin(train_lst)]  

# 100% CRISPR for Train
dftrain2=df_proceed[df_proceed['Metadata_experiment_type']=='CRISPR']    
#dftrain2.to_csv('/data/datacenter/H3C_GPU/projects/yuchen/gzsda-main/gzsda-main/data/XrayBaggage20/total_CRISPR.csv')
# To meet the mAP evaluation Plate
train_part_total=pd.DataFrame()
for i in lst:
    train_part_total=pd.concat([train_part_total,dftrain1])#.loc[dftrain1['Metadata_Plate']==i,:]])
train_part0=pd.DataFrame()
for i in ['BR00116991','BR00116992','BR00116993','BR00116994']:
    train_part0=pd.concat([train_part0,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part1=pd.DataFrame()
for i in ['BR00117015','BR00117016','BR00117017','BR00117019']:
    train_part1=pd.concat([train_part1,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part2=pd.DataFrame()
for i in ['BR00116995','BR00117024','BR00117025','BR00117026']:
    train_part2=pd.concat([train_part2,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part3=pd.DataFrame()
for i in ['BR00117010','BR00117011','BR00117012','BR00117013']:
    train_part3=pd.concat([train_part3,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])  
    
# test data sampling
gene_seen=pd.DataFrame()
for i in train_lst:
    train_part0_filtered=train_part0.loc[train_part0['Metadata_gene']==i,:]
    train_part1_filtered=train_part1.loc[train_part1['Metadata_gene']==i,:]
    train_part2_filtered=train_part2.loc[train_part2['Metadata_gene']==i,:]
    train_part3_filtered=train_part3.loc[train_part3['Metadata_gene']==i,:]

# To filter seen/unseen gene from compound perturbation

    if len(train_part0_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part0_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part0_filtered[train_part0_filtered['Metadata_pert_iname']==filter_out]])
        
    if len(train_part1_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part1_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part1_filtered[train_part1_filtered['Metadata_pert_iname']==filter_out]]) 
        
    if len(train_part2_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part2_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part2_filtered[train_part2_filtered['Metadata_pert_iname']==filter_out]])  
        
    if len(train_part3_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part3_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part3_filtered[train_part3_filtered['Metadata_pert_iname']==filter_out]])
train_part_total_final=train_part_total.drop(index=list(gene_seen['Unnamed: 0'].index))
train_part_total_final=train_part_total_final.drop_duplicates()
df_test1=gene_seen
df_test1=df_test1[df_test1['Metadata_Plate'].isin(lst)]
# train
df_train=pd.concat([train_part_total_final,dftrain2]) 
df_test2=df_proceed[df_proceed['Metadata_gene'].isin(test_lst)]
df_test2=df_test2.drop_duplicates()
df_test2=df_test2[df_test2['Metadata_experiment_type']=='Compound']
df_test2=df_test2[df_test2['Metadata_Plate'].isin(lst)]
# SEEN

# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
df_test1_rm_emptygene=df_test1[df_test1['Metadata_gene']!='empty']
y_test=df_test1_rm_emptygene[df_test1_rm_emptygene['Metadata_experiment_type']=='Compound']['Metadata_gene']
testX=df_test1_rm_emptygene[df_test1_rm_emptygene['Metadata_experiment_type']=='Compound'].drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)
df_proceed = df_proceed[df_proceed['Metadata_Plate'].isin(lst)]

In [ ]:
k=130
CP_seen_result= kNCM(trainX, trainY, testX, testY, k,'top_10')
CP_seen_result['If_seen']='seen'
CP_seen_result1= kNCM(trainX, trainY, testX, testY, k,'top_1')
CP_seen_result1['If_seen']='seen'

In [ ]:
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
df_test2_rm_emptygene=df_test2[df_test2['Metadata_gene']!='empty']
y_test=df_test2_rm_emptygene[df_test2_rm_emptygene['Metadata_experiment_type']=='Compound']['Metadata_gene']
testX=df_test2_rm_emptygene[df_test2_rm_emptygene['Metadata_experiment_type']=='Compound'].drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)

In [ ]:
CP_unseen_result= kNCM(trainX, trainY, testX, testY, k,'top_10')
CP_unseen_result['If_seen']='unseen'
CP_total_result=pd.concat([CP_seen_result,CP_unseen_result])
print('CP_KNCM_SEEN_TOP10',CP_seen_result['score'].mean())
print('CP_KNCM_UNEEN_TOP10',CP_unseen_result['score'].mean())
print('CP_KNCM_HARMONIC_MEAN_TOP10',2/((1/CP_seen_result['score'].mean())+ 1 /CP_unseen_result['score'].mean() ))
CP_unseen_result1= kNCM(trainX, trainY, testX, testY, k,'top_1')
CP_unseen_result1['If_seen']='unseen'
CP_total_result1=pd.concat([CP_seen_result1,CP_unseen_result1])
print('')
print('CP_KNCM_SEEN_TOP1',CP_seen_result1['score'].mean())
print('CP_KNCM_UNEEN_TOP1',CP_unseen_result1['score'].mean())
print('CP_KNCM_HARMONIC_MEAN_TOP1',2/((1/CP_seen_result1['score'].mean())+ 1 /CP_unseen_result1['score'].mean() ))

In [ ]:
import math
def calculate_std(inputs):
    n = len(inputs)
    mean = sum(inputs) / n
    deviations = [(x - mean) ** 2 for x in inputs]
    variance = sum(deviations) / (n - 1)
    std = math.sqrt(variance)
    return std

In [ ]:
inputs= [0.1215, 0.1121, 0.1039, 0.1009, 0.0904]
sum(inputs)/5

In [ ]:
calculate_std(inputs)

In [ ]:
2/((1/0.16975 )+ (1 / 0.29512499999999997) )

In [ ]:
2/((1/0.22262500000000002 )+ (1 / 0.317625  ) )

In [ ]:
2/((1/0.2035  )+ (1 /0.35450000000000004 ) )

In [ ]:
2/((1/0.28925)+ (1 /0.27549999999999997 ) )

In [ ]:
2/((1/0.181625)+ (1 /0.319125) )

In [ ]:
import math
def calculate_std(inputs):
    n=len(inputs)
    mean = sum(inputs)/n
    deviations = [(x-mean)**2 for x in inputs]
    variance = sum(deviations)/(n-1)
    std = math.sqrt(variance)
    return std

In [ ]:
inputs=[0.21553092229093843,0.26177238546968995,0.25856899641577064,0.2822076139884905,0.23149706689965055]
sum(inputs)/5

In [ ]:
calculate_std(inputs)